## Evaluation Step (with Gemini Pro)

In [1]:
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Image,
    Part,
    HarmBlockThreshold,
    HarmCategory,
)

In [8]:
import os

# Set the environment variable before running the script. You can set it in the terminal or in the code.
# os.environ["BUCKET_UPLOAD_TEMP"] = "<your-bucket>"
# bucket_name = os.environ.get("BUCKET_UPLOAD_TEMP")


In [9]:
from google.cloud import storage

def upload_file_to_temp_bucket(file_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    blob = bucket.blob(file_name)
    blob.upload_from_filename(file_name)
    
    return blob.public_url.replace("https://storage.googleapis.com/", "gs://")

In [10]:
gemini_model = GenerativeModel("gemini-pro-vision")


In [25]:
def generate_catch_phrases(business_goal, local_image_resource):
  output_json = """
  { "phrases": ["당신에게 더 많은 활력을. 스피디 테니스화", "코트와의 일체감. 스피디 테니스화", "테니스공보다 빠른 운동화", "만들기가 어려운 테니스화", "테니스계의 이단아"] }
  """
  prompt_template = """You are a copy writer for a telecom company. You need to write 5 short descriptions(copies) for a given business goal and image in json format

  [output examples]
  {output_json}

  [business goal]
  {business_goal}

  [image]
  """

  generation_config = GenerationConfig(
      temperature=0.1,
      max_output_tokens=2048,
  )

  safety_config = {
      HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
  }

  image_uri = upload_file_to_temp_bucket(local_image_resource)

  prompt = prompt_template.format(output_json=output_json, business_goal = business_goal)
  image = Part.from_uri(
      uri=image_uri,
      mime_type="image/jpeg",
  )
  contents = [prompt, image]

  responses = gemini_model.generate_content(contents, generation_config=generation_config, 
      safety_settings=safety_config, stream=False)

  return responses

In [26]:
responses = generate_catch_phrases("10대 20대를 위한 키치한 판매 문구. 최소 10자 이상. 30자 미만", 'resources/black_shoes_on_the_beach.jpeg')

In [27]:
responses

candidates {
  content {
    role: "model"
    parts {
      text: " { \"phrases\": [\"가볍고 편안한 스니커즈\", \"젊음을 위한 최고의 신발\", \"활동적인 라이프 스타일을 위한 완벽한 신발\", \"최신 트렌드를 따르는 스타일리시한 신발\", \"친구와 어울리거나 외출하기에 좋은 신발\"] }"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 422
  candidates_token_count: 76
  total_token_count: 498
}

In [28]:
import json
result_list = json.loads(responses.candidates[0].content.parts[0].text)


In [29]:
result_list

{'phrases': ['가볍고 편안한 스니커즈',
  '젊음을 위한 최고의 신발',
  '활동적인 라이프 스타일을 위한 완벽한 신발',
  '최신 트렌드를 따르는 스타일리시한 신발',
  '친구와 어울리거나 외출하기에 좋은 신발']}

In [30]:
for phrase in result_list['phrases']:
  print(phrase)

가볍고 편안한 스니커즈
젊음을 위한 최고의 신발
활동적인 라이프 스타일을 위한 완벽한 신발
최신 트렌드를 따르는 스타일리시한 신발
친구와 어울리거나 외출하기에 좋은 신발


In [31]:

def verify_phrase(phrase):

  generation_config = GenerationConfig(
      temperature=0.1,
      max_output_tokens=2048,
  )

  safety_config = {
      HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
  }
  
  verifier_prompt = """You are the final marketing copy reviewer. Please check if the given copy has any potential issues when used as a marketing copy - e.g., racism, ageism, sexism - and express your score on it, between 0 and 100 points.

  [phrase]
  {phrase}
  """
  contents = [verifier_prompt.format(phrase = phrase)]
  responses = gemini_model.generate_content(contents, generation_config=generation_config, safety_settings=safety_config, stream=False)
  return responses

In [33]:
for phrase in result_list['phrases']:
  responses = verify_phrase(phrase)
  print(f"phrase: {phrase}, score: {responses.candidates[0].content.parts[0].text}")
  

phrase: 가볍고 편안한 스니커즈, score: The given phrase does not contain any potential issues such as racism, ageism, or sexism. It is a neutral and descriptive phrase that simply describes the product's features.

Score: 100/100
phrase: 젊음을 위한 최고의 신발, score: **Potential Issues:**

* **Ageism:** The phrase implies that only young people can wear the shoes, which could be seen as excluding older adults.

**Score:** 50/100

**Recommendation:**

To address the ageism issue, the phrase could be revised to something like:

> "The best shoes for people of all ages"
phrase: 활동적인 라이프 스타일을 위한 완벽한 신발, score: The given copy does not contain any potential issues related to racism, ageism, or sexism. It is a neutral statement that describes the product's intended use.

Score: 100/100
phrase: 최신 트렌드를 따르는 스타일리시한 신발, score: The given phrase does not contain any potential issues related to racism, ageism, or sexism. It is a neutral statement that describes the product's style.

Score: 100/100
phrase: 친구와 어울리거나 외

In [34]:
def check_creativity(phrase):
  generation_config = GenerationConfig(
      temperature=0.1,
      max_output_tokens=2048,
  )

  safety_config = {
      HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
  }
  
  verifier_prompt = """You are the final marketing copy reviewer. Please express, on a scale of 0 (low) to 100 (high), whether the given sentence is creative and conveys enough meaning to customers when used as a marketing copy.

  [phrase]
  {phrase}
  """
  contents = [verifier_prompt.format(phrase = phrase)]
  responses = gemini_model.generate_content(contents, generation_config=generation_config, safety_settings=safety_config, stream=False)
  return responses

In [35]:
for phrase in result_list['phrases']:
  responses = check_creativity(phrase)
  print(f"phrase: {phrase}, score: {responses.candidates[0].content.parts[0].text}")  

phrase: 가볍고 편안한 스니커즈, score: 70
phrase: 젊음을 위한 최고의 신발, score: 70
phrase: 활동적인 라이프 스타일을 위한 완벽한 신발, score: 70
phrase: 최신 트렌드를 따르는 스타일리시한 신발, score: 75
phrase: 친구와 어울리거나 외출하기에 좋은 신발, score: 50


## For Text Bison

In [36]:
from vertexai.language_models import (
    TextGenerationModel,
    TextEmbeddingModel,
    ChatModel,
    InputOutputTextPair,
    CodeGenerationModel,
    CodeChatModel,
)

In [38]:
text_model_old_api = TextGenerationModel.from_pretrained("text-bison@002")


In [43]:
def generate_catch_phrases_by_textbison(business_goal, target_product_description):
  output_json = """
  { "phrases": ["당신에게 더 많은 활력을. 스피디 테니스화", "코트와의 일체감. 스피디 테니스화", "테니스공보다 빠른 운동화", "만들기가 어려운 테니스화", "테니스계의 이단아"] }
  """
  prompt_template = """You are a copy writer for a retail company in Korea. You need to write 5 short descriptions(copies) for a given business goal and product description in json format

  [output examples]
  {output_json}

  [business goal]
  {business_goal}

  [ad target product description]
  {target_product_description}
  """

  prompt = prompt_template.format(output_json=output_json, business_goal = business_goal, target_product_description = target_product_description)
  parameters = {
      "temperature": 0.2,
      "max_output_tokens": 2048,  # Token limit determines the maximum amount of text output.
      "top_p": 0.8,  # Tokens are selected from most probable to least until the sum of their probabilities equals the top_p value.
      "top_k": 40,  # A top_k of 1 means the selected token is the most probable among all tokens.
  }
  responses = text_model_old_api.predict(prompt=prompt, **parameters)

  return responses.text

In [47]:
results = generate_catch_phrases_by_textbison("10대 20대를 위한 키치한 판매 문구. 최소 10자 이상. 30자 미만", "바삭바삭한 질감이 일품인 포테이토 칩")

In [48]:
print(results)

 { "phrases": ["바삭한 맛이 일품인 포테이토 칩", "한 입 베어 물면 멈출 수 없는 포테이토 칩", "바삭한 맛이 입안 가득 퍼지는 포테이토 칩", "한 번 먹으면 잊을 수 없는 포테이토 칩", "바삭한 맛이 넘치는 포테이토 칩"] }


In [49]:
result_json = json.loads(results)


In [51]:
def verify_phrase_by_textbison(phrase):

  verifier_prompt = """You are the final marketing copy reviewer. Please check if the given copy has any potential issues when used as a marketing copy - e.g., racism, ageism, sexism - and express your score on it, between 0 and 100 points.

  [phrase]
  {phrase}
  """

  parameters = {
      "temperature": 0.2,
      "max_output_tokens": 2048,  # Token limit determines the maximum amount of text output.
      "top_p": 0.8,  # Tokens are selected from most probable to least until the sum of their probabilities equals the top_p value.
      "top_k": 40,  # A top_k of 1 means the selected token is the most probable among all tokens.
  }
  responses = text_model_old_api.predict(prompt=verifier_prompt.format(phrase=phrase), **parameters)
  return responses.text

In [53]:
for phrase in result_json['phrases']:
  responses = verify_phrase_by_textbison(phrase)
  print(f"phrase: {phrase}, score: {responses}")


phrase: 바삭한 맛이 일품인 포테이토 칩, score:  **Score: 90/100**

The phrase "바삭한 맛이 일품인 포테이토 칩" is a marketing copy for potato chips. It describes the product as having an excellent crispy taste. There are no potential issues of racism, ageism, or sexism in this phrase. It is a simple and straightforward description of the product.
phrase: 한 입 베어 물면 멈출 수 없는 포테이토 칩, score:  **Score: 85/100**

The phrase "한 입 베어 물면 멈출 수 없는 포테이토 칩" is a marketing copy for potato chips. It means "Potato chips that you can't stop eating once you take a bite." There are no obvious issues of racism, ageism, or sexism in the phrase. However, it may be considered slightly suggestive as it implies that the potato chips are so delicious that people will lose control and keep eating them. This could be seen as a negative aspect of the product, as it may lead to overeating and weight gain. Overall, the phrase is effective in conveying the message that the potato chips are delicious and addictive, but it may need to be revised